In [394]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
from datetime import date
import calendar

In [395]:
# importe les données dans un dataframe
birth_df = pd.read_csv("birth.csv", sep=",")

In [396]:
birth_df.head(3)
print("Nombre total de lignes :", birth_df.index.size)
print("Nombre de non nul par colonne :",birth_df.count())  # count compte seulmt les valeurs renseignées de chaque colonne (!= de .size, qui compte tout)
# En comptant les éléments par colonne on s'aperçoit que seuls certains jours ne sont pas renseignés

Nombre total de lignes : 15547
Nombre de non nul par colonne : year      15547
month     15547
day       15067
gender    15547
births    15547
dtype: int64


In [389]:
print("Nbe de lignes, avant nettoyage :" , birth_df.index.size)

# élimination des lignes dont l'année, le mois ou le jour n'est pas renseigné
birth_df.dropna(subset = ['year','month','day'], inplace=True)

# on garde seulement les lignes dont les années sont entre 1900 et 2019
#cond_y = pd.Series((birth_df.year < 2019)&(birth_df.year > 1900))

# on garde seulement les lignes dont les mois sont compris entre 1 et 12
#cond_m = pd.Series([birth_df.month[i] in range (1,13) for i in range(len(birth_df.month))])

# on garde seulement les lignes dont les jours sont compris entre 1 et 31
cond_d = pd.Series([birth_df.day.values[i] in range (1,32) for i in range(len(birth_df.day))], \
                   index = birth_df.day.index)

birth_df = birth_df.where(cond_d).dropna()

# nbe de lignes après triage
print("Nbe de lignes, après nettoyage :", birth_df.index.size)

Nbe de lignes, avant nettoyage : 15547
Nbe de lignes, après nettoyage : 14717


In [233]:
df = pd.DataFrame({'year': [2015, 2016],'month': [2, 3],'day': [4.0, 5.0]})
pd.to_datetime(df) # type "Series"

0   2015-02-04
1   2016-03-05
dtype: datetime64[ns]

In [216]:
birth_df[-125:-120]

,year,month,day,gender,births
14942,1988,10,31.0,F,4893
14943,1988,10,31.0,M,5086
14944,1988,10,99.0,M,1
14945,1988,11,1.0,F,5826
14946,1988,11,1.0,M,6036


In [219]:
# Crée une dataframe contenant les données pour former les dates (123 de la fin, 63 du début)
date_df = pd.DataFrame({'year':birth_df.year.values,'month':birth_df.month.values,'day':birth_df.day.values})
pd.to_datetime(date_df)#,infer_datetime_format=True)
#date_ser = pd.Series(pd.to_datetime(date_df,infer_datetime_format=True))
#date_df

ValueError: cannot assemble the datetimes: 'float' object is unsliceable

In [218]:
my_date = date.today()
print(my_date)
print(my_date.weekday())
print(calendar.day_name[0])
jours_tab = [calendar.day_name[date_tab[i].weekday()] for i in range(len(date_tab))]
jours_df = pd.DataFrame(jours_tab, columns = ['day_week']) 

2019-03-08
4
Monday


NameError: name 'date_tab' is not defined

In [219]:
birth_df_bis = pd.concat([birth_df, jours_df], 1)
#mask1 = birth_df_bis['day_week']=='Friday'
mask2 = birth_df_bis['year']==1982
mask3 = birth_df_bis['month']==6
mask4 = birth_df_bis['day']==11
birth_df_bis[mask2][mask3][mask4]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,year,month,day,gender,births,day_week
10207,1982,6,11.0,F,5159,Sunday
10208,1982,6,11.0,M,5470,Sunday


In [ ]:
birth_df_bis.groupby

In [ ]:

pan_vnon80 = fam_pan_df[fam_pan_df["ventre"]]
